In [1]:
import sys
import os
sys.path.append(os.path.abspath('../src'))
sys.path.append(os.path.abspath('../Dataset'))
sys.path.append(os.path.abspath('..'))

import importlib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

from func_preprocesamiento import preprocesar, preprocesar_binarios, procesar_antiguedad, preprocesar_categoricos

In [2]:
dev_df = pd.read_csv('../Dataset/raw/dev.csv')


train_dataset, val_dataset = train_test_split(dev_df, train_size=0.2, random_state=42)
X_train = train_dataset.drop(columns=['precio_pesos_constantes'])
y_train = train_dataset['precio_pesos_constantes']

C:\Users\Camii\AppData\Local\Temp\ipykernel_8160\3635508779.py:1: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  dev_df = pd.read_csv('../Dataset/raw/dev.csv')


In [3]:
df = preprocesar(X_train)
direccion = '../Dataset/Procesado/dev_procesado.csv'
df.to_csv(direccion, index=False)

ind_cols inicializada: ['Dormitorios', 'Banos', 'Ambientes', 'Cocheras', 'STotalM2', 'SConstrM2']
ind_cols en preprocesar_categoricos: ['Dormitorios', 'Banos', 'Ambientes', 'Cocheras', 'STotalM2', 'SConstrM2']
Columna a predecir: ITE_ADD_NEIGHBORHOOD_NAME_encoded
Nan en X: Dormitorios    0
Banos          0
Ambientes      0
Cocheras       0
STotalM2       0
SConstrM2      0
dtype: int64
Número de filas con valores faltantes en la columna objetivo: 2208
Número de filas sin valores faltantes en la columna objetivo: 45170
bin
Columna a predecir: SalonDeUsosMul
Nan en X: Dormitorios                          0
Banos                                0
Ambientes                            0
Cocheras                             0
STotalM2                             0
SConstrM2                            0
ITE_ADD_CITY_NAME_encoded            0
ITE_ADD_STATE_NAME_encoded           0
ITE_ADD_NEIGHBORHOOD_NAME_encoded    0
dtype: int64
Número de filas con valores faltantes en la columna objetivo: 6

ValueError: Input X contains NaN.
RandomForestClassifier does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [ ]:

columna = ['ITE_ADD_CITY_NAME', 'ITE_ADD_STATE_NAME', 'ITE_ADD_NEIGHBORHOOD_NAME']
X_train = preprocesar_categoricos(X_train, columna)


         ITE_ADD_CITY_NAME  ITE_ADD_CITY_NAME_encoded
73778      Capital Federal                          4
83752           La Matanza                         15
81714   Esteban Echeverría                          8
171550     Capital Federal                          4
189337          San Miguel                         28
...                    ...                        ...
119879     Capital Federal                          4
103694     Capital Federal                          4
131932          La Matanza                         15
146867     Capital Federal                          4
121958          La Matanza                         15

[47383 rows x 2 columns]


In [ ]:
columna = 'Antiguedad'
df_pr_antig = procesar_antiguedad(X_train)
print(df_pr_antig[columna].value_counts())

Antiguedad
0       19570
40       3033
10       2428
50       1892
5        1682
        ...  
105         1
2017        1
2009        1
2002        1
98          1
Name: count, Length: 153, dtype: int64


In [ ]:
columna = 'Amoblado'
print(X_train[columna])
print(X_train['Amoblado'].value_counts())

73778      No
83752      No
81714       0
171550      0
189337      0
         ... 
119879     No
103694     No
131932    NaN
146867      0
121958      0
Name: Amoblado, Length: 47383, dtype: object
Amoblado
No      19961
0       12446
0.0      5360
Sí       2319
1.0       776
   0      610
1         437
Name: count, dtype: int64


In [ ]:
#O: 38377 + 5474 FALTANTES = 43851
col_binarias_np = ['Amoblado']
col_binarias_imp = ['Amoblado']
binario_procesado = preprocesar_binarios(X_train, ['Amoblado'], 'RF')
#print(binario_procesado['Amoblado'].value_counts())

Vector real: [0. 1. 0. ... 0. 0. 0.]
Vector predicho: [0. 1. 0. ... 0. 0. 0.]
RMSE: 0.27329975745967033
